In [21]:
import numpy as np
import pandas as pd
import ast
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
import torch

In [2]:
#df = pd.read_csv('train.csv')

## SOME INSIGHTS OF THE DATASET
#### This data set consists of 3 columns: dialog, act and emotion. The dialog column has one conversation, consisting of multiple dialogs. And for each sentence we a classification of it's act and emotion. Secondly, the dtype of all the columns are object and they are string of list of string ie, "['xyz', 'abc', 'pqr']". Similarly for the other columns.

In [3]:
#df.info()

In [4]:
#df.iloc[0].dialog

In [5]:
#df.iloc[0].act

In [6]:
#df.iloc[0].emotion

In [7]:
'''import re

def parse_dialog(dialog_str):
    if not isinstance(dialog_str, str):
        return []
    matches = re.findall(r"'(.*?)'|\"(.*?)\"", dialog_str)
    return [m[0] or m[1] for m in matches]


def parse_labels(label_str):
    if not isinstance(label_str, str):
        return []
    return list(map(int, label_str.strip("[]").split()))


def flatten_dialog_dataset(df):
    rows = []

    for _, row in df.iterrows():
        sentences = parse_dialog(row["dialog"])
        acts = parse_labels(row["act"])
        emotions = parse_labels(row["emotion"])

        if not (len(sentences) == len(acts) == len(emotions)):
            continue  # skip broken rows

        for s, a, e in zip(sentences, acts, emotions):
            rows.append({
                "sentence": s.strip(),
                "act": a,
                "emotion": e
            })

    return pd.DataFrame(rows)'''

'import re\n\ndef parse_dialog(dialog_str):\n    if not isinstance(dialog_str, str):\n        return []\n    matches = re.findall(r"\'(.*?)\'|"(.*?)"", dialog_str)\n    return [m[0] or m[1] for m in matches]\n\n\ndef parse_labels(label_str):\n    if not isinstance(label_str, str):\n        return []\n    return list(map(int, label_str.strip("[]").split()))\n\n\ndef flatten_dialog_dataset(df):\n    rows = []\n\n    for _, row in df.iterrows():\n        sentences = parse_dialog(row["dialog"])\n        acts = parse_labels(row["act"])\n        emotions = parse_labels(row["emotion"])\n\n        if not (len(sentences) == len(acts) == len(emotions)):\n            continue  # skip broken rows\n\n        for s, a, e in zip(sentences, acts, emotions):\n            rows.append({\n                "sentence": s.strip(),\n                "act": a,\n                "emotion": e\n            })\n\n    return pd.DataFrame(rows)'

In [8]:
#new_df = flatten_dialog_dataset(df)

#new_df["sentence"] = new_df["sentence"].str.lower().apply(wordpunct_tokenize)

In [9]:
!pip install gensim

In [10]:
'''from gensim.test.utils import common_texts
from gensim.models import Word2Vec'''

'from gensim.test.utils import common_texts\nfrom gensim.models import Word2Vec'

In [11]:
#corpus = new_df['sentence'].tolist()
#model = Word2Vec(sentences=corpus, vector_size=256, window=7, sg=1, min_count=1, epochs=18, sample = 1e-3, negative=5, workers=4)

In [12]:
#model.wv.save("word2vec_256.kv")

In [13]:
'''from gensim.models import KeyedVectors
wv = KeyedVectors.load("word2vec_256.kv")'''

'from gensim.models import KeyedVectors\nwv = KeyedVectors.load("word2vec_256.kv")'

In [14]:
#wv.most_similar('tired', topn=10)

In [15]:
#new_df

In [16]:
import torch
import torch.nn as nn

In [17]:
'''def build_sentence_embeddings(new_df, wv):
  def sentence_to_matrix(tokens):
    vectors = []
    for token in tokens:
      if token in wv:
        vectors.append(wv[token])
      #else:
        #vectors.append(np.zeros(embedding_dim))

    return np.vstack(vectors)

  new_df['embedding_matrix'] = new_df['sentence'].apply(sentence_to_matrix)

  return new_df'''

"def build_sentence_embeddings(new_df, wv):\n  def sentence_to_matrix(tokens):\n    vectors = []\n    for token in tokens:\n      if token in wv:\n        vectors.append(wv[token])\n      #else:\n        #vectors.append(np.zeros(embedding_dim))\n\n    return np.vstack(vectors)\n\n  new_df['embedding_matrix'] = new_df['sentence'].apply(sentence_to_matrix)\n\n  return new_df"

In [18]:
#df = build_sentence_embeddings(new_df, wv)

In [19]:
#df.iloc[0].embedding_matrix

In [20]:
'''def positional_encodings(max_len, d_model=256):

  PE = np.zeros((max_len, d_model))

  for pos in range(max_len):
    for i in range(d_model//2):

      PE[pos, 2*i] = np.sin((pos) / (10000**(2*i / 512)))
      PE[pos, 2*i+1] = np.cos((pos) / (10000**(2*i / 512)))

  return PE'''

'def positional_encodings(max_len, d_model=256):\n\n  PE = np.zeros((max_len, d_model))\n\n  for pos in range(max_len):\n    for i in range(d_model//2):\n\n      PE[pos, 2*i] = np.sin((pos) / (10000**(2*i / 512)))\n      PE[pos, 2*i+1] = np.cos((pos) / (10000**(2*i / 512)))\n\n  return PE'

In [21]:
'''df['sentence'].apply(len).describe()'''

"df['sentence'].apply(len).describe()"

In [22]:
'''PE = positional_encodings(max_len=294, d_model=256)
df['embedding_matrix'] = df['embedding_matrix'].apply(lambda x: x + PE[:x.shape[0]])'''

"PE = positional_encodings(max_len=294, d_model=256)\ndf['embedding_matrix'] = df['embedding_matrix'].apply(lambda x: x + PE[:x.shape[0]])"

In [23]:
#df

In [24]:
#df.to_pickle("dataset_with_embeddings.pkl")

In [ ]:
'''import torch
import numpy as np
df['embedding_matrix'] = df['embedding_matrix'].apply(lambda x: torch.from_numpy(x))'''

In [ ]:
#df.to_pickle("dataset_with_embeddings.pkl")

In [32]:
df

,sentence,act,emotion,embedding_matrix
0,"[say, ,, jim, ,, how, about, going, for, a, fe...",3,0,"[[tensor(-0.1942, dtype=torch.float64), tensor..."
1,"[you, know, that, is, tempting, but, is, reall...",4,0,"[[tensor(0.0734, dtype=torch.float64), tensor(..."
2,"[what, do, you, mean, ?, it, will, help, us, t...",2,0,"[[tensor(-0.0327, dtype=torch.float64), tensor..."
3,"[do, you, really, think, so, ?, i, don, ', t, ...",2,0,"[[tensor(-0.0939, dtype=torch.float64), tensor..."
4,"[i, guess, you, are, right, ., but, what, shal...",2,0,"[[tensor(-0.0713, dtype=torch.float64), tensor..."
...,...,...,...,...
86786,"[i, want, a, pair, of, locus, .]",3,0,"[[tensor(-0.0713, dtype=torch.float64), tensor..."
86787,"[take, a, look, at, the, ones, on, display, ,,...",4,0,"[[tensor(-0.3741, dtype=torch.float64), tensor..."
86788,"[i, need, size, 41, .]",3,0,"[[tensor(-0.0713, dtype=torch.float64), tensor..."
86789,"[could, i, have, the, check, ,, please, ?]",3,0,"[[tensor(-0.2140, dtype=torch.float64), tensor..."


In [ ]:
#df['sentence'].apply(len).describe()

In [26]:
import torch.nn.utils.rnn
from torch.nn.utils.rnn import pad_sequence

In [4]:
df = pd.read_pickle("dataset_with_embeddings.pkl")

In [22]:
class SentenceDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.sentences = df["embedding_matrix"].tolist()
        self.labels = df['emotion'].tolist()

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx], self.labels

In [23]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # batch = [(sentence_tensor, label), ...]

    sentences, labels = zip(*batch)  # unzip
    
    lengths = torch.tensor([x.shape[0] for x in sentences])  # (B,)
    
    # Pad sentence tensors
    padded = pad_sequence(batch, batch_first=True)  # (B, L, d_model)
    
    # Create attention mask
    max_len = padded.shape[1]
    mask = torch.arange(max_len).expand(len(sentences), max_len)
    mask = mask >= lengths.unsqueeze(1)
    mask = mask.unsqueeze(1).unsqueeze(2)  # (B,1,1,L)
    mask = mask.float() * -1e9

    labels = torch.tensor(labels, dtype=torch.long)
    
    return padded, mask, labels

In [30]:
from torch.utils.data import DataLoader

dataset = SentenceDataset(df)

loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)